In [2]:
import requests
import pandas as pd
import datetime
from datetime import timedelta
from datetime import timezone
import time

In [3]:
#pool = '0x39AA39c021dfbaE8faC545936693aC917d5E7563' #cUSDC
#pool = '0xf650C3d88D12dB855b8bf7D11Be6C55A4e07dCC9' #cUSDT
#pool = '0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643' #cDAi
#pool = '0x4Ddc2D193948926D02f9B1fE9e1daa0718270ED5' #cETH
#pool = 'xc11b1268c1a384e55c48c2391d8d480264a3a7f4' #cWBTC - not
#pool = '0x35A18000230DA775CAc24873d00Ff85BccdeD550' #cUNI
pool = '0xFAce851a4921ce59e912d19329929CE6da6EB0c7' #cLINK




date_1 = int(datetime.datetime(2022, 8, 30, 0,0).replace(tzinfo=timezone.utc).timestamp())
date_2 = int(datetime.datetime(2022, 8, 31, 0,0).replace(tzinfo=timezone.utc).timestamp())

In [4]:
lst = []

In [5]:
def process_compound(raw):
    epoch = raw['supply_rates'][0]['block_timestamp']
    time = datetime.datetime.utcfromtimestamp(epoch)
    supply = raw['supply_rates'][0]['rate']
    borrow = raw['borrow_rates'][0]['rate']
        
    lst.append(['', epoch, time, supply, borrow])

In [6]:
cont = True

while cont:
    query = f'https://api.compound.finance/api/v2/market_history/graph?asset={pool}&num_buckets=1&min_block_timestamp={date_1}&max_block_timestamp={date_2}'
    response = requests.get(query)
    raw = response.json()
    process_compound(raw)
    date_1 = date_1 - 86400
    date_2 = date_2 - 86400
    print(datetime.datetime.utcfromtimestamp(date_2).strftime('%d-%m-%Y'))
    time.sleep(3)


30-08-2022
29-08-2022
28-08-2022
27-08-2022
26-08-2022
25-08-2022
24-08-2022
23-08-2022
22-08-2022
21-08-2022
20-08-2022
19-08-2022
18-08-2022
17-08-2022
16-08-2022
15-08-2022
14-08-2022
13-08-2022
12-08-2022
11-08-2022
10-08-2022
09-08-2022
08-08-2022
07-08-2022
06-08-2022
05-08-2022
04-08-2022
03-08-2022
02-08-2022
01-08-2022
31-07-2022
30-07-2022
29-07-2022
28-07-2022
27-07-2022
26-07-2022
25-07-2022
24-07-2022
23-07-2022
22-07-2022
21-07-2022
20-07-2022
19-07-2022
18-07-2022
17-07-2022
16-07-2022
15-07-2022
14-07-2022
13-07-2022
12-07-2022
11-07-2022
10-07-2022
09-07-2022
08-07-2022
07-07-2022
06-07-2022
05-07-2022
04-07-2022
03-07-2022
02-07-2022
01-07-2022
30-06-2022
29-06-2022
28-06-2022
27-06-2022
26-06-2022
25-06-2022
24-06-2022
23-06-2022
22-06-2022
21-06-2022
20-06-2022
19-06-2022
18-06-2022
17-06-2022
16-06-2022
15-06-2022
14-06-2022
13-06-2022
12-06-2022
11-06-2022
10-06-2022
09-06-2022
08-06-2022
07-06-2022
06-06-2022
05-06-2022
04-06-2022
03-06-2022
02-06-2022
01-06-2022

IndexError: list index out of range

In [7]:
df_compound = pd.DataFrame(lst, columns=['asset', 'epoch', 'timestamp', 'supply', 'borrow'])
df_compound['asset'] = 'link'
df = df_compound.copy()
df = df.sort_values(by='timestamp', ascending=True)
df = df.reset_index(drop=True)
#store in csv
df.to_csv(f'data/compound_link_interest.csv', index=False)